In [1]:
pip install numpy torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


We've implemented linear regression & gradient descent model using some basic tensor operations. However, since this is a common pattern in deep learning, PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.

In [2]:
import torch.nn as nn
import numpy as np 
import torch

PyTorch provides several built-in functions and classes to make it easy to create and train models with just a few lines of code.
Let's begin by importing the torch.nn package from PyTorch, which contains utility classes for building neural networks.
We are using 15 training examples to illustrate how to work with large datasets in small batches.

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

We'll create a TensorDataset, which allows access to rows from inputs and targets as tuples, and provides standard APIs for working with many different types of datasets in PyTorch
The TensorDataset allows us to access a small section of the training data using the array indexing notation ([0:3] in the above code)

In [4]:
from torch.utils.data import TensorDataset

In [5]:
# define datasets
train_ds=TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

We'll also create a DataLoader, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data

In [6]:
from torch.utils.data import DataLoader

# define data loader
batch_size=5
train_dl=DataLoader(train_ds, batch_size, shuffle=True)

In [10]:
for xb ,yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 87., 135.,  57.],
        [ 88., 134.,  59.],
        [ 73.,  66.,  44.],
        [ 69.,  96.,  70.],
        [ 68.,  96.,  71.]])
tensor([[118., 134.],
        [118., 132.],
        [ 57.,  69.],
        [103., 119.],
        [104., 118.]])


 Suffle can be used to randomized the input and otimizes the output leading to faster reduction in loss.
 previously we can add weight and baises manually, but their we can use build in function like nn.linear class from pytorch

In [13]:
# define model
model=nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.2214,  0.5714,  0.0151],
        [ 0.1531, -0.4687, -0.3840]], requires_grad=True)
Parameter containing:
tensor([ 0.1195, -0.5608], requires_grad=True)


In [14]:
# generate prediction as we do 
predict=model(inputs)
predict

tensor([[ 22.8947, -37.2946],
        [ 31.2275, -52.4443],
        [ 58.3080, -72.3113],
        [  2.6696, -19.3014],
        [ 40.7601, -61.8669],
        [ 22.1019, -36.6728],
        [ 30.6712, -52.3597],
        [ 58.1018, -72.5422],
        [  3.4624, -19.9232],
        [ 40.9966, -62.4041],
        [ 22.3384, -37.2099],
        [ 30.4347, -51.8225],
        [ 58.8643, -72.3959],
        [  2.4331, -18.7642],
        [ 41.5529, -62.4887]], grad_fn=<AddmmBackward0>)

In [15]:
# we can use buildin function for the loss function
# Import nn.functional
import torch.nn.functional as F

In [16]:
# The nn.functional package contains many useful loss functions and several other utilities
# Define loss function
loss_fn = F.mse_loss

# lets compute the loss
loss=loss_fn(model(inputs),targets)
print(loss)

tensor(12453.9648, grad_fn=<MseLossBackward0>)


In [21]:
# for optimization
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

# Train The Model

In [22]:
# Utility function to train the model
def train(num_epochs, model, loss_fn, opt, train_dl):
    for epoch in range(num_epochs):
        # train with batches of data
        for xb, yb in train_dl:

            # make prediction
            pred=model(xb)

            # make loss function
            loss=loss_fn(pred,yb)

            #compute gradient
            loss.backward()

            # update gradient
            opt.step()

            #reset gradient

            opt.zero_grad()

            #print the progess

        if (epoch+1)%10==0:
            print('Epoch [{}/{}], Loss: {:.2f}'.format(epoch+1, num_epochs, loss.item()))


In [23]:
train(200,model, loss_fn, opt, train_dl)

Epoch [10/200], Loss: 366.53
Epoch [20/200], Loss: 511.62
Epoch [30/200], Loss: 273.55
Epoch [40/200], Loss: 153.73
Epoch [50/200], Loss: 99.52
Epoch [60/200], Loss: 85.40
Epoch [70/200], Loss: 71.19
Epoch [80/200], Loss: 55.19
Epoch [90/200], Loss: 44.23
Epoch [100/200], Loss: 32.77
Epoch [110/200], Loss: 26.92
Epoch [120/200], Loss: 15.28
Epoch [130/200], Loss: 17.15
Epoch [140/200], Loss: 25.04
Epoch [150/200], Loss: 24.89
Epoch [160/200], Loss: 9.82
Epoch [170/200], Loss: 35.50
Epoch [180/200], Loss: 25.44
Epoch [190/200], Loss: 20.06
Epoch [200/200], Loss: 12.35


In [24]:
predict=model(inputs)
print(predict)

tensor([[ 57.3560,  70.9991],
        [ 80.0707,  97.9642],
        [122.5432, 138.6527],
        [ 22.4021,  40.7326],
        [ 97.5081, 112.1037],
        [ 56.0737,  69.8966],
        [ 79.5403,  97.5238],
        [122.6283, 138.9783],
        [ 23.6844,  41.8352],
        [ 98.2601, 112.7659],
        [ 56.8256,  70.5587],
        [ 78.7884,  96.8616],
        [123.0736, 139.0931],
        [ 21.6502,  40.0705],
        [ 98.7905, 113.2063]], grad_fn=<AddmmBackward0>)


In [25]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])
